In [1]:
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from sklearn.model_selection import train_test_split

from model import MusicTransformer
import torch
import torch.nn as nn
import torch.optim as optim

/var/folders/95/0csrhky965x_1zggpg7jfkr00000gn/T/ipykernel_44131/3752955255.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
chords = pd.read_csv("output_chords.csv", header=None)
# chords.head

chords_sorted = chords.apply(lambda x: sorted(x), axis=1)

# Drop duplicates to get unique chords
# Count the unique chords
# vocab_size = len(chords_sorted.drop_duplicates())
# print("Vocabulary Size (Unique Chords):", vocab_size)
# # vocab is 10950 for /content/output_chords.csv
vocab_size = 10950


In [4]:

import numpy as np
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch

batch_size=128

# Load the dataset
dataset = pd.read_csv('output_chords.csv')

def create_sequences(dataset, sequence_length=4):
    input_sequences = []
    output_notes = []

    for i in range(round(len(dataset)*1 - sequence_length)):
        sequence = dataset[i:i + sequence_length].values.tolist()

        # Selecting 2 random notes from the 4th chord
        fourth_chord = sequence[-1]
        output = random.sample(fourth_chord, 2)

        # Removing the selected notes from the 4th chord in the input
        for note in output:
            fourth_chord.remove(note)

        input_sequences.append(sequence)
        output_notes.append(output)

    return input_sequences, output_notes

# Create input-output pairs
input_sequences, output_notes = create_sequences(dataset)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(input_sequences, output_notes, test_size=0.2, random_state=42,shuffle=False)

# Preparing PyTorch dataset
class ChordDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        # Prepare the input sequence as a 4x3 matrix
        sequence_matrix = []
        for chord in self.sequences[idx]:
            # Pad the chord with zeros if less than 3 notes
            padded_chord = chord + [0] * (3 - len(chord))
            sequence_matrix.append(padded_chord)

        return {
            'sequence': torch.tensor(sequence_matrix, dtype=torch.long),
            'label': torch.tensor(self.labels[idx], dtype=torch.float32)
        }

# Adjust the create_sequences function if needed to ensure correct formatting


# Creating data loaders for training and validation
train_dataset = ChordDataset(X_train, y_train)
val_dataset = ChordDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# The train_loader and val_loader are now ready to be used for training and validation


In [34]:


# Assuming the definition of the MusicTransformer model is available from model.py
# Initialize the Music Transformer model
note_vocab_size = 128  # As specified earlier
embed_size = 384  # Example size, adjust as needed
num_layers = 6  # Example value, adjust as needed
heads = 6  # Example value, adjust as needed
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
forward_expansion = 4  # Example value, adjust as needed
dropout = 0.3 # Example dropout rate, adjust as needed
max_length = 100  # Maximum sequence length, adjust as needed
epochs=30
learning_rate= 0.0001


model = MusicTransformer(
    note_vocab_size,
    embed_size,
    num_layers,
    heads,
    device,
    forward_expansion,
    dropout,
    max_length
).to(device)


# for name, param in model.named_parameters():
#   print(f"{name}: {param.size()}")

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001)

# Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1):
    model.train()
    for epoch in range(epochs):
        for batch in train_loader:
            optimizer.zero_grad()

            # Extract data and move tensors to the appropriate device
            input_data = batch['sequence'].to(device)
            targets = batch['label'].to(device)
            # Ensure the tensor is of integer type
            targets = targets.long()  # Convert to long if not already an integer type


            # Select the appropriate column for one-hot encoding
            # Adjust this based on your data's structure

            one_hot_col1 = F.one_hot(targets[:, 0], num_classes=128)
            one_hot_col2 = F.one_hot(targets[:, 1], num_classes=128)

            # Concatenate along the last dimension
            one_hot_combined = torch.cat((one_hot_col1, one_hot_col2), dim=-1)

            one_hot_final = one_hot_combined[:, :128]
            # Check if indices are in the valid range

            outputs = model(input_data, None)  # Assuming no mask for simplicity
            # out1, out2 = outputs.split();
            loss = criterion(outputs, one_hot_final.float())
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

        # Validation Loop
        model.eval()
        with torch.no_grad():
            for batch in val_loader:
                input_data = batch['sequence'].to(device)
                targets = batch['label'].to(device)
                targets = targets.long()
                one_hot_col1 = F.one_hot(targets[:, 0], num_classes=128)
                one_hot_col2 = F.one_hot(targets[:, 1], num_classes=128)

                # Concatenate along the last dimension
                one_hot_combined = torch.cat((one_hot_col1, one_hot_col2), dim=-1)

                one_hot_final = one_hot_combined[:, :128]

                outputs = model(input_data, None)
                val_loss = criterion(outputs, one_hot_final.float())

                # Here, you can also calculate accuracy or other metrics as needed

        print(f"Validation Loss: {val_loss.item():.4f}")

    # Save the model
    print('saving model_state_dict')

    hyperparameters = {
      'learning_rate': learning_rate,
      'batch_size': batch_size,
      'num_epochs': epochs,
      'embed_size': embed_size,
      'num_layers': num_layers,
      'heads': heads,
      'forward_expansion': forward_expansion,
      'dropout': dropout,
      'max_length': max_length,
      'note_vocab_size': note_vocab_size,
    }

    # Combine the model's state dict and the hyperparameters in a single dictionary
    save_dict = {
        'model_state_dict': model.state_dict(),
        'hyperparameters': hyperparameters
    }


    torch.save(save_dict, 'model_state_dict.pth')

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=epochs)


cpu
Epoch [1/1], Loss: 3.2473
Validation Loss: 3.4490
saving model_state_dict
